In [59]:
import importlib

import numpy as np
import pandas as pd
import numpy
import lab_graph

importlib.reload(lab_graph)

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

df = pd.read_csv("data/all-colors-unique.csv")

#Create a 2D array of Lab values
lab_list = list(df["L*a*b Value"])
lab_list = [x.split(", ") for x in lab_list]
lab_points = [[float(x) for x in row] for row in lab_list]

#Use greedy approach
result = lab_graph.greedy_mdp(lab_points, 16)

#Create new dataframe with just the resultant indices
result_df = df.loc[result]

#lab_graph.generate_lab_3d_all_colors(result_df, "Test 1", "N/a")

In [60]:
from scipy.spatial.distance import pdist, squareform
import colormath.color_objects as co
import colormath.color_diff as cdiff

distances = squareform(pdist(lab_points))
#Retrieves the indices furthest points
i, j = np.unravel_index(np.argmax(distances), distances.shape)

print("___Euclidean Distance___")
print(df.iloc[i])
print(df.iloc[j])
# Use lab points

def lab_point_distance(color1, color2, delta_e="CIE2000"):
    """
    Compute the difference between two colors using a Delta E formula.

    :param color1: Lab color number 1
    :param color2: Lab color number 2
    :param delta_e: The comparison formula, CIE2000 (symmetric) is default
    :return: Returns the distance between two colors using the CIE2000 or CMC delta E formula
    """
    lab_1 = co.LabColor(color1[0], color1[1], color1[2])
    lab_2 = co.LabColor(color2[0], color2[1], color2[2])

    if delta_e == "CIE2000":
        return cdiff.delta_e_cie2000(lab_1, lab_2, Kl=2, Kc=1, Kh=1)
    if delta_e == "CMC":
        return cdiff.delta_e_cmc(lab_1, lab_2, pl=2, pc=1)

distances = squareform(pdist(lab_points, metric=lab_point_distance))
i,j = np.unravel_index(np.argmax(distances), distances.shape)

print("\n___Delta_E_Distance___")
print(df.iloc[i])
print(df.iloc[j])


___Euclidean Distance___
Color Name                      Poppy
PMS Color Code                  2347C
L*a*b Value       49.45, 75.12, 67.21
Hex                            E10600
Name: 36, dtype: object
Color Name                         Teal
PMS Color Code                    3262C
L*a*b Value       66.44, -59.33, -12.05
Hex                              00BFB2
Name: 46, dtype: object

___Delta_E_Distance___
Color Name                      Berry
PMS Color Code                  233CP
L*a*b Value       45.38, 71.84, -7.30
Hex                            C6057B
Name: 6, dtype: object
Color Name               Heather Kelly
PMS Color Code                   340 C
L*a*b Value       51.80, -65.36, 14.55
Hex                             00965E
Name: 72, dtype: object


In [61]:
# Need a scoring function
# Take a set of lab point indices



# Score by average distance
# Score by Convex Hull

#Average distance
#Provide a pair of points from

,Color Name,PMS Color Code,L*a*b Value,Hex
32,Olive,5747CP,"31.03, -6.88, 23.88",474C21
64,Heather Columbia Blue,660 U,"56.78, 0.45, -38.70",658BCC
35,Pink,706C,"86.80, 16.25, 1.44",F7CED7
100,Heather Team Purple,2665 C,"45.45, 34.51, -52.90",7D55C7
36,Poppy,2347C,"49.45, 75.12, 67.21",E10600


In [ ]:
#Retrieve the hundred furthest points
flat_indices = np.argpartition(distances.flatten(), -100)[-100:]
# Convert to 2D indices
i_indices, j_indices = np.unravel_index(flat_indices, distances.shape)
# Get corresponding distances
top_distances = distances[i_indices, j_indices]
# Sort by distance (optional)
sort_idx = np.argsort(top_distances)[::-1]
i_indices = i_indices[sort_idx]
j_indices = j_indices[sort_idx]
top_distances = top_distances[sort_idx]

print(j_indices[0], i_indices[0])
print(j_indices[1], i_indices[1])